In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from RobotEnv import RobotEnv
import random
env = RobotEnv(screen_width=400, screen_height=400)
# size of obs
observation_length = env.observation_space.shape[0]
action_length = env.action_space.n

# 定义模型

In [ ]:
import torch


class SelectOutput(torch.nn.Module):
    def forward(self, inputs):
        outputs, (hidden, cell) = inputs
        return outputs


model = torch.nn.Sequential(
    torch.nn.Linear(observation_length, 64),
    torch.nn.ReLU(),
    torch.nn.LSTM(64, 64, batch_first=True),
    SelectOutput(),  # 添加自定义层
    torch.nn.ReLU(),
    torch.nn.Linear(64, action_length),
)
model_delay = torch.nn.Sequential(
    torch.nn.Linear(observation_length, 64),
    torch.nn.ReLU(),
    torch.nn.LSTM(64, 64, batch_first=True),
    SelectOutput(),  # 添加自定义层
    torch.nn.ReLU(),
    torch.nn.Linear(64, action_length),
)
# 复制参数
model_delay.load_state_dict(model.state_dict())

# 单局游戏

In [ ]:
# 定义优化器
from utils import Controller,Pool
controller = Controller(model, env)
pool = Pool(controller)
# pool.update()
# pool.sample()

# 训练

In [26]:
from torch.utils.tensorboard import SummaryWriter
import os
import json
import numpy as np


# 训练


def train():

    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)
    loss_fn = torch.nn.MSELoss()
    # 共训练n_step次
    n_step = 0
    log_interval = 100_000
    last_log_step = 0
    writer = SummaryWriter("./logs/DRQN_logs")
    # 如果checkpoint存在，则加载checkpoint
    if os.path.exists("checkpoint/DRQN.pth"):
        model.load_state_dict(torch.load("checkpoint/DRQN.pth"))
        model_delay.load_state_dict(torch.load("checkpoint/DRQN.pth"))
    if os.path.exists("checkpoint/DRQN_logs.json"):
        with open('checkpoint/DRQN_logs.json', 'r') as f:
            log_data = np.array(json.load(f))
            n_step = log_data[:, 1][-1]
            last_log_step = n_step
            for row in log_data:
                writer.add_scalar('Step', row[1], global_step=row[1])
                writer.add_scalar('Test Result', row[2], global_step=row[1])
    while n_step < 100_000_000:
        n_step += pool.update()
        # print(f"n_step:{n_step}")
        # 每次更新数据后,训练N次
        for i in range(200):
            # print(f"i:{i}")
            # 采样N条数据
            state, action, reward, next_state, terminated = pool.sample()

            # 计算value
            value = model(state).gather(dim=1, index=action)

            # 计算target
            with torch.no_grad():
                # 使用原模型计算动作,使用延迟模型计算target,进一步缓解自举
                next_action = model(next_state).argmax(dim=1, keepdim=True)
                target = model_delay(next_state).gather(dim=1,
                                                        index=next_action)
            target = target * 0.99 * (1 - terminated) + reward

            loss = loss_fn(value, target)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # 复制参数
        if (n_step - last_log_step) >= log_interval:
            model_delay.load_state_dict(model.state_dict())
            test_result = sum([pool.controller.play(mode="test")[-1]
                              for _ in range(20)]) / 20
            print(f"step:{n_step},test_result:{test_result}")

            last_log_step = n_step
            # 将步数，测试结果和损失写入TensorBoard
            writer.add_scalar('Step', n_step, global_step=n_step)
            writer.add_scalar('Test Result', test_result, global_step=n_step)
            writer.add_scalar('Loss', loss.item(), global_step=n_step)
    writer.close()  # 训练结束后关闭writer
train()

KeyboardInterrupt: 

In [ ]:
# # 保存模型参数
# torch.save({
#     'model_state_dict': model.state_dict(),
#     # 'optimizer_state_dict': optimizer.state_dict(),
#     'n_step': n_step,
# }, 'DRQN.pth')